# Generate multimodal travel time matrix (ttm)

This script is to generate a multimodal ttm for accessibility calculation in _r5py_.<br/>
Modes: walking, biking, transit, car

## 1. Variables definition and data import

In [1]:
# definitions
import sys
import numpy as np
import pandas as pd
import geopandas as gpd
import datetime as dt
import tracc
from r5py import TransportNetwork, TravelTimeMatrixComputer, TransitMode, LegMode
from datetime import datetime,date,timedelta
import matplotlib.pyplot as plt
sys.argv.append(["--max-memory", "8G"])


data_folder = "/Users/azanchetta/OneDrive - The Alan Turing Institute/demoland_data"


# regional level files: (require previous editing)
oas_centroids_file = f"{data_folder}/processed/OA_centroids_TyneWear.gpkg" # used for population origin
oas_file = f"{data_folder}/processed/authorities/OA_TyneWear.gpkg" # needed for visualisation purposes
region_lads_file = f"{data_folder}/processed/authorities/LADs_tynewear.shp" # needed in order to filter greenspace data within the regional boundaries
workingplacezones_centroids_file = f"{data_folder}/processed/authorities/WPZ_centroids_tynewear.gpkg"

# national level files
greenspace_file = f"{data_folder}/raw/accessibility/OS Open Greenspace (GPKG) GB/data/opgrsp_gb.gpkg"
osm_data_file = f"{data_folder}/raw/accessibility/tyne-and-wear-latest.osm.pbf"
gtfs_data_file = f"{data_folder}/raw/accessibility/itm_north_east_gtfs.zip"

In [2]:
# import

# origins (IE output areas, OAs)
oas_centroids = gpd.read_file(oas_centroids_file,
                              layer="OA_centroids_TyneWear")
oas_centroids['id'] = oas_centroids['OA11CD'] # Origin dataset must contain an 'id' column
oas_centroids_wgs84 = oas_centroids.to_crs("epsg:4326")
oas_centroids.head()

# destination data
# green space sites' entrances
gs_entrances = gpd.read_file(greenspace_file,
                        layer = "AccessPoint")

gs_entrances.head()
# WPZ centroids
wpz_centroids = gpd.read_file(workingplacezones_centroids_file,
                              layer = "WPZ_centroids_tynewear")
wpz_centroids.head()
wpz_centroids['id'] = wpz_centroids['wz11cd'] # Destination dataset must contain an 'id' column

# network data
# uploaded in the sequent operation

In [4]:
# for testing purposes
# selecting from origin and destinations only a few points
# IE selecting first n rows
# n=99
# origins = oas_centroids.loc[:n, :]
# destinations = wpz_centroids.loc[:n, :]
origins = oas_centroids
destinations = wpz_centroids

### CRS conversion

Converting original files crs to crs compatible with GTFS and OSM data

In [5]:
origins.head()
origins.crs #epsg:27700
origins = origins.to_crs("epsg:4326")
origins.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
origins.explore()

In [6]:
destinations.head()
destinations.crs #epsg:27700
destinations = destinations.to_crs("epsg:4326")
destinations.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
destinations.explore()

## 2. ttm generation and more

### Generate the transport network from OSM and GTFS data

In [7]:
# load in transport network
transport_network = TransportNetwork(
    osm_data_file,
    [
        gtfs_data_file
    ]
)

### Create an empty matrix that contains all origins and destinations to be used later on

This table will be filled up once we calculate the ttm

In [8]:
# destinations
# gs: entrances + oas centroids
# jobs: wpz centroids + oas centroids

from itertools import product
# generate dataframe with all from_id and all to_id pairs
prod = product(origins['id'].unique(),
               destinations['id'].unique())
empty_ttm = pd.DataFrame(prod)
empty_ttm.columns = ['from_id', 'to_id']
empty_ttm.head()

,from_id,to_id
0,E00041377,E33000251
1,E00041377,E33000799
2,E00041377,E33000257
3,E00041377,E33000079
4,E00041377,E33000174


### Travel time matrix

In [ ]:
# # Trials for ttm computation
# trial_comp = TravelTimeMatrixComputer(
#     transport_network,
#     origins=origins,
#     destinations=destinations,
#     departure=dt.datetime(2023,1,19,8,30),
#     # max_time = dt.timedelta(seconds=900),
#     transport_modes=[LegMode.CAR, TransitMode.TRANSIT]
# )
# trial = trial_comp.compute_travel_times()
# trial.travel_time.isna().sum()

The following piece of code is split in 2:
- first part is definition of variables that will be inputted as parameters in the ttm computation
- second part is the loop to generate ttm for several transport modes

In [11]:
# defining variables
date_time = '2023,01,19,9,30'
max_time = dt.timedelta(seconds=900) # we don't use this currently, we'll filter the times later on
walking_speed = 4.8
cycling_speed = 16
# dataframe to match legmode and transitmode objects (to be inputted in the ttm computer):
modes_lut = pd.DataFrame([
                          ['transit', TransitMode.TRANSIT, LegMode.WALK],
                          ['car', '', LegMode.CAR],
                          ['bicycle', '', LegMode.BICYCLE],
                          ['walk','', LegMode.WALK],
                         ],
                         columns = ('Mode', 'Transit_mode', 'Leg_mode'))

# function to generate custom list of transit+transport mode for the parameter transport_modes in TravelTimeMatrixComputer
def list_making(s,z):
    return [s] + [z]

ttm_complete = empty_ttm

# loop to compute a ttm for all the modes and generate one single ttm table in output
for row in modes_lut.itertuples():
    start_time = dt.datetime.now()
    mode = row.Mode
    transit_mode = row.Transit_mode
    leg_mode = row.Leg_mode
    transport_mode = list_making(transit_mode,leg_mode) # creating list of objects for transport_modes parameter

    print('The current mode is:', mode, ', transit is:', transit_mode, ', transport var is:', transport_mode)
    ttm_computer = TravelTimeMatrixComputer(
        transport_network,
        origins = origins,
        destinations = destinations,
        departure = dt.datetime.strptime(date_time, '%Y,%m,%d,%H,%M'),
        speed_walking = walking_speed,
        speed_cycling = cycling_speed,
        transport_modes = transport_mode
    )

    ttm = ttm_computer.compute_travel_times()
    ttm = ttm.rename(columns = {'travel_time':f'time_{mode}'}) # renaming 'travel_time' column (automatically generated) to 'time_{mode of transport}'
    ttm.isna().sum() # checking for empty values, to see if the ttm actually calculated something
    #  merging the empty table generated before (with all possible origins and destinations) with the ttm, per each mode adding a travel time column
    ttm_complete = ttm_complete.merge(ttm,
                    how ='outer',
                    left_on = ['from_id','to_id'],
                    right_on = ['from_id','to_id'])
    
    print('finished calculating ttm for mode', mode)
    end_time = datetime.now()
    print('Duration for', mode, ': {}'.format(end_time - start_time))

The current mode is: transit , transit is: TransitMode.TRANSIT , transport var is: [<TransitMode.TRANSIT: <java object 'com.conveyal.r5.api.util.TransitModes'>>, <LegMode.WALK: <java object 'com.conveyal.r5.api.util.LegMode'>>]


/usr/local/anaconda3/envs/demoland_r5/lib/python3.9/site-packages/r5py/r5/regional_task.py:224: RuntimeWarning: Departure time 2023-01-19 09:30:00 is outside of the time range covered by currently loaded GTFS data sets.
  warnings.warn(


finished calculating ttm for mode transit
Duration for transit : 0:06:35.415242
The current mode is: car , transit is:  , transport var is: ['', <LegMode.CAR: <java object 'com.conveyal.r5.api.util.LegMode'>>]
finished calculating ttm for mode car
Duration for car : 0:18:50.111351
The current mode is: bicycle , transit is:  , transport var is: ['', <LegMode.BICYCLE: <java object 'com.conveyal.r5.api.util.LegMode'>>]
finished calculating ttm for mode bicycle
Duration for bicycle : 0:14:20.641112
The current mode is: walk , transit is:  , transport var is: ['', <LegMode.WALK: <java object 'com.conveyal.r5.api.util.LegMode'>>]
finished calculating ttm for mode walk
Duration for walk : 0:02:27.494674


In [ ]:
# %reset_selective -f empty_ttm